In [1]:
import numpy as np
import pandas as pd
import pandasql as ps

import warnings
warnings.filterwarnings('ignore')

 ![](computers.png)

In [84]:
product = pd.read_csv('sample/product.csv', index_col='model')
product.iloc[:4]

,maker,type
model,,
1121,B,PC
1232,A,PC
1233,A,PC
1260,E,PC


In [85]:
laptop = pd.read_csv('sample/laptop.csv', index_col='code')
laptop.iloc[:4]

,model,speed,ram,hd,price,screen
code,,,,,,
1,1298,350,32,4.0,700,11
2,1321,500,64,8.0,970,12
3,1750,750,128,12.0,1200,14
4,1298,600,64,10.0,1050,15


In [86]:
pc = pd.read_csv('sample/pc.csv', index_col='code')
pc.iloc[:4]

,model,speed,ram,hd,cd,price
code,,,,,,
1,1232,500,64,5.0,12x,600
2,1121,750,128,14.0,40x,850
3,1233,500,64,5.0,12x,600
4,1121,600,128,14.0,40x,850


In [87]:
printer = pd.read_csv('sample/printer.csv', index_col='code')
printer[:4]

,model,color,type,price
code,,,,
1,1276,n,Laser,400
2,1433,y,Jet,270
3,1434,y,Jet,290
4,1401,n,Matrix,150


### task 1
Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость. 

### sql

In [88]:
query = '''
    select
        product.maker, laptop.speed
    from product join laptop 
    on product.model = laptop.model and laptop.hd >= 10
    '''
ps.sqldf(query)

,maker,speed
0,B,750
1,A,600
2,A,750
3,A,450


### task 1
Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость. 

### pandas

In [89]:
product_laptop = product.merge(laptop, left_index=True, right_on='model')
product_laptop

,maker,type,model,speed,ram,hd,price,screen
code,,,,,,,,
1,A,Laptop,1298,350,32,4.0,700,11
4,A,Laptop,1298,600,64,10.0,1050,15
6,A,Laptop,1298,450,64,10.0,950,12
2,C,Laptop,1321,500,64,8.0,970,12
3,B,Laptop,1750,750,128,12.0,1200,14
5,A,Laptop,1752,750,128,10.0,1150,14


In [90]:
product_laptop[product_laptop['hd'] >= 10][['maker', 'speed']]

,maker,speed
code,,
4,A,600
6,A,450
3,B,750
5,A,750


### task 2

Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.

### pandas

In [91]:
maker = product[product['type']=='PC'].groupby('maker').count()
maker = maker.rename(columns = {'type':'count'})
maker

,count
maker,
A,2
B,1
E,3


In [92]:
maker[maker['count'] >= 3]

,count
maker,
E,3


### task 2

Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.

### pandas

In [93]:
query = '''
    select
        maker,
        count(model) as count
    from product
    where type = 'PC'
    group by maker
    having count >= 3
    '''
data = ps.sqldf(query)
data.set_index('maker')

,count
maker,
E,3


### task 3
Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price 

### pandas

In [94]:
data = product[product['type']=='Printer'][['maker']].merge(
    printer[printer['color']=='y'], left_index=True, right_on='model')
data

,maker,model,color,type,price
code,,,,,
2,D,1433,y,Jet,270
3,E,1434,y,Jet,290


In [95]:
pd.DataFrame([data.min()])[['maker', 'price']]

,maker,price
0,D,270


### task 3
Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price 

### sql

In [96]:
query = '''
    select
        maker,
        min(price) as price
    from product join printer
        on product.model = printer.model and printer.color = 'y'
'''
ps.sqldf(query)

,maker,price
0,D,270


### task 4
Найдите модели ПК-блокнотов, скорость которых меньше скорости любого из ПК. Вывести: model, speed.

### sql

In [97]:
query = '''
    select
        laptop.model,
        laptop.speed
    from laptop join pc
    on laptop.speed < (select min(pc.speed) from pc)
    group by laptop.model
'''
ps.sqldf(query)

,model,speed
0,1298,350


### task 4
Найдите модели ПК-блокнотов, скорость которых меньше скорости любого из ПК. Вывести: model, speed.

### pandas

In [98]:
laptop[laptop['speed'] < pc['speed'].min()][['model', 'speed']]

,model,speed
code,,
1,1298,350


### task 5

Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM.

### sql

In [99]:
query = '''
    select
        a.model,
        b.model,
        a.speed,
        a.ram
    from pc as a join pc as b
    on a.speed = b.speed and a.ram = b.ram and a.model > b.model
'''
ps.sqldf(query)

,model,model,speed,ram
0,1233,1232,500,64
1,1233,1121,750,128
2,1260,1232,500,32


### task 5

Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM.

### pandas

In [100]:
data = pc.merge(pc, left_on=['speed','ram'], right_on=['speed', 'ram'])
data.iloc[:4]

,model_x,speed,ram,hd_x,cd_x,price_x,model_y,hd_y,cd_y,price_y
0,1232,500,64,5.0,12x,600,1232,5.0,12x,600
1,1232,500,64,5.0,12x,600,1233,5.0,12x,600
2,1233,500,64,5.0,12x,600,1232,5.0,12x,600
3,1233,500,64,5.0,12x,600,1233,5.0,12x,600


In [101]:
data[(data['model_x'] > data['model_y'])][['model_x', 'model_y', 'speed', 'ram']]

,model_x,model_y,speed,ram
2,1233,1232,500,64
6,1233,1121,750,128
14,1260,1232,500,32


### task 6

Найти производителей, которые выпускают более одной модели, при этом все выпускаемые производителем модели являются продуктами одного типа.
Вывести: maker, type 

### pandas

In [102]:
more_one = product.groupby('maker').count() > 1
more_one

,type
maker,
A,True
B,True
C,False
D,True
E,True


In [103]:
unique = product.groupby(['maker']).nunique()[['type']] == 1
unique

,type
maker,
A,False
B,False
C,True
D,True
E,False


In [104]:
idxs = product[['maker']].merge(
    more_one & unique, right_index=True, left_on='maker')['type']

In [105]:
product.loc[idxs].drop_duplicates()

,maker,type
model,,
1288,D,Printer


### task 6

Найти производителей, которые выпускают более одной модели, при этом все выпускаемые производителем модели являются продуктами одного типа.
Вывести: maker, type 

### sql

In [106]:
query = '''
    select maker, type
    from product
    group by maker
    having count(distinct type) = 1 and count(model) > 1
'''
ps.sqldf(query)

,maker,type
0,D,Printer


### task 7

Задача по предсказанию, сколько потратит клиент в свой первый визит на следующей неделе.

### pandas

In [107]:
# load data
train = pd.read_csv('sample/data.csv')
train[4:8]

,id,date,sum
4,1,15,8
5,1,17,6
6,1,23,2
7,1,24,7


In [108]:
# weights by date for weighted mode 
train['weights'] = (train.date / train.date.max()) ** 1.2
train[4:8]

,id,date,sum,weights
4,1,15,8,0.017440
5,1,17,6,0.020266
6,1,23,2,0.029128
7,1,24,7,0.030654


In [109]:
byw = train.groupby(['id', 'sum'])[['weights']].sum()
byw.loc[1:1]

weights
id sum           
1  1    26.984579
   2    14.208423
   3     5.630956
   4     5.456149
   5    11.246348
   6    15.971761
   7    18.317880
   8     5.361063

In [110]:
ind = byw.reset_index().groupby('id')['weights'].apply(np.argmax)
ind[:4]

id
1     0
2    11
3    16
4    24
Name: weights, dtype: int64

In [111]:
prediction = byw.iloc[ind]
prediction[:4]

,,weights
id,sum,
1,1,26.984579
2,4,19.809903
3,5,10.248162
4,6,13.086052


In [112]:
prediction.reset_index('sum', inplace=True)
prediction[:4]

,sum,weights
id,,
1,1,26.984579
2,4,19.809903
3,5,10.248162
4,6,13.086052


In [113]:
# calculate last day of visit
train['delta_day'] = train['date'].diff()
train[:4]

,id,date,sum,weights,delta_day
0,1,2,6,0.001554,NaN
1,1,3,7,0.002528,1.0
2,1,8,5,0.008202,5.0
3,1,10,7,0.010721,2.0


In [114]:
# calculate mean last day
prediction['delta_day'] = train.groupby('id')['delta_day'].mean()
prediction[:4]

,sum,weights,delta_day
id,,,
1,1,26.984579,2.164179
2,4,19.809903,-0.040650
3,5,10.248162,-1.520833
4,6,13.086052,0.905882


In [115]:
# calculate mean day between visits
prediction['last_day'] = train.groupby('id').tail(1).reset_index().set_index('id')['date']
prediction[:4]

,sum,weights,delta_day,last_day
id,,,,
1,1,26.984579,2.164179,437
2,4,19.809903,-0.040650,432
3,5,10.248162,-1.520833,359
4,6,13.086052,0.905882,436


In [116]:
# parameters
f_delta = -4
f_minus = -7
f_plus = 7
last_day = 438

# filter: long time without visits
# filter: estimated visit before our week
# filter: estimated visit after our week
def make_zero(row):
    if ((row['last_day'] - last_day < f_minus) 
            or (row['last_day'] + row['delta_day'] - last_day < f_delta)
            or (row['last_day'] + row['delta_day'] - last_day > f_plus)):
        return 0
    else:
        return row['sum']

prediction['sum'] = prediction.apply(make_zero, axis=1)
prediction[:4]

,sum,weights,delta_day,last_day
id,,,,
1,1.0,26.984579,2.164179,437
2,0.0,19.809903,-0.040650,432
3,0.0,10.248162,-1.520833,359
4,6.0,13.086052,0.905882,436


In [117]:
# saving
prediction[['sum']].to_csv('sample/submission.csv')